<a href="https://colab.research.google.com/github/EllaThomasson/labs-ds3001/blob/main/LinRegLab_EllaThomasson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Question 0**

Part 1

A linear model should be linear in Betas.

Part 2

Part 3

Yes, linear regression can be used for classification but it is generally not advised because of interpretation and assumption violations. Logistic regression shoukd gerenally be used for classification.

Part 4

If a linear model is overfitting, the training set will perform much better than the testing set.

Part 5

Part 6

Part 7